In [57]:
import matplotlib.pyplot as plt
import math
import numpy
import pandas as pd
import time


import src.tools as tools
import src.recencytools as recency
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load files

In [58]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

path_to_results = 'results/'

Beta is the coefficient that regulates time discrepancy.
The higher beta is, the more importance is given to older messages.

Each mail is weighted according to its rank (order of anciency).

In [59]:
beta = 6000
training_info = recency.add_time_rank_to_dataframe(training_info) # Run only once !
email_ranks_dic = recency.get_email_ranks(training_info)

In [60]:
# Compute score for a given mid
some_mid = 378257
recency.get_email_recency_score(email_ranks_dic, some_mid, beta)

0.10389927764229338

Compute the recency address book.

In [61]:
email_ids_per_sender = preprocess.get_email_ids_per_sender(training)
recency_address_books = recency.get_recency_address_books(email_ids_per_sender, training_info, beta)

processed 0 senders
processed 10 senders
processed 20 senders
processed 30 senders
processed 40 senders
processed 50 senders
processed 60 senders
processed 70 senders
processed 80 senders
processed 90 senders
processed 100 senders
processed 110 senders
processed 120 senders


In [62]:
# Write recency predictions to file
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)
recency_predictions = recency.predictions_from_addressbook(test_email_ids_per_sender, recency_address_books)
recency_predictions = recency.recency_predictions_to_standard(recency_predictions)
postprocess.write_results_ranked(recency_predictions, path_to_results,
                                 'predictions_recency_{beta}.txt'.format(beta=beta))